In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**content**

* introduction

* overview of the data set

* logistic regression
  * computation graph
  * initializing parameters
  * forward propagation
    * sigmoid function
    * loss function
    * cost function
  * optimazation algorithm with gradient descent
    * backward propagation
    * updating parameters
  * logistic regression with sklearn


* artificial neural network
  * 2 layer neural network
    * size of layers and initializing paramaeters weights and bias
    * forward propagation
    * loss function and cost function
    * update parameters
    * prediction with learnt parameters weights and bias
    * create model
  * l layer neural network
    * implementing with keras library


We will use "sign language digits data set" for this tutorial.

In this data there are 2062 sign language digits images.

As you know digits are from 0 to 9. Therefore there are 10 unique sign.

At the beginning of tutorial we will use only sign 0 and 1 for simplicity.

In data, sign zero is between indexes 204 and 408. Number of zero sign is 205.

Also sign one is between indexes 822 and 1027. Number of one sign is 206. Therefore, we will use 205 samples from each classes(labels).

Note: Actually 205 sample is very very very little for deep learning. But this is tutorial so it does not matter so much.
Lets prepare our X and Y arrays. X is image array (zero and one signs) and Y is label array (0 and 1).

In [12]:
# load data set
x_l = np.load('/kaggle/input/sign-language-digits-dataset/X.npy')
Y_l = np.load('/kaggle/input/sign-language-digits-dataset/Y.npy')

img_size = 64
plt.subplot(1, 2, 1)
plt.imshow(x_l[260].reshape(img_size, img_size))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(x_l[900].reshape(img_size, img_size))
plt.axis('off')

In [15]:
# join a sequence of arrays along an row axis
X = np.concatenate((x_l[204:409], x_l[822:1027] ), axis=0)
# from 0 to 204 is zero sign and from 205 to 410 is one sign
z = np.zeros(205)
o = np.ones(205)
Y = np.concatenate((z, o), axis=0).reshape(X.shape[0],1)
print(" X shape: " , X.shape)
print(" Y shape: " , Y.shape)

the shape of the x is (410, 64, 64)
410 means that we have 412 images (zero and one signs)
64 mean that our image size is 64x64 pixels

the shape of the Y is (410,1)
410 means that we have 410 labels(0 and 1)

let split X and Y into train and test sets
percentage of test size, test = 15% and tarin = 75%

random_state = use same seed while randomizing, it always creates same test and tarin set if we call split function repeatedly.

In [26]:
# Then lets create x_train, y_train, x_test, y_test arrays
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]


now we have 3D input array X, we need to make it flatten(2D) in order to use as input for iur first deep learning model

our label array (Y) is a already flatten(2D) 


In [27]:
X_train_flatten = X_train.reshape(number_of_train,X_train.shape[1]*X_train.shape[2])
X_test_flatten = X_test .reshape(number_of_test,X_test.shape[1]*X_test.shape[2])
print("X train flatten",X_train_flatten.shape)
print("X test flatten",X_test_flatten.shape)


we have 348 images and each image has 4096 pixels in image train array

we have 62 images and each image has 4096 pixel in images test arays

now we transpose

transpose() function changes the row elements into column elements and the column elements into row elements. The output of this function is a modified array of the original one.

In [28]:
x_train = X_train_flatten.T
x_test = X_test_flatten.T
y_train = Y_train.T
y_test = Y_test.T

print("x train: ", x_train.shape)
print("x test: ", x_test.shape)
print("y train: ", y_train.shape)
print("y test: ", y_test.shape)

so far what we did

choose our label that were zero sign one

create and flatten train and test sets

then tranpose the falttened arrays

**logistic regression**
 we use it for binary calssification
 
**initializinf parameters**

our imaghe ahs 4096 pixel in x_train

each pixels have own weights

the firts steo is multiplying each pixels with their own weights

weight are 0.01 and the weight array shape is (4096,1)

initial bias is 0



In [41]:
# initialize parameters
def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1), 0.01)
    b = 0.0
    return w, b


Forward Propagation

The all steps from pixels to cost is called forward propagation

z = (w.T)x + b => in this equation we know x that is pixel array, we know w (weights) and b (bias) so the rest is calculation. (T is transpose)

Then we put z into sigmoid function that returns y_head(probability). When your mind is confused go and look at computation graph. Also equation of sigmoid function is in computation graph.

Then we calculate loss(error) function.

Cost function is summation of all loss(error).

Lets start with z and the write sigmoid definition(method) that takes z as input parameter and returns y_head(probability)

In [42]:
# calculation of z
# z =np.dot(w.T, x_train)+b
def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

In [31]:
y_head = sigmoid(0)
y_head

In [43]:
# forward propagation steps
# find z = w.T*x+b
# y_head = sigmoid(z)
# loss(error) = loss(y,y_head)
# cost = sum(loss)

def forward_propagation(w,b,x_train,y_train):
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1] # x_train.shape is for scaling
    return cost

Optimization Algorithm with Gradient Descent

now we know what is our cost that is error.

Therefore, we need to decrease cost because as we know if cost is high it means that we make wrong prediction.

Lets think first step, every thing starts with initializing weights and bias. Therefore cost is dependent with them.

In order to decrease cost, we need to update weights and bias.

In other words, our model needs to learn the parameters weights and bias that minimize cost function. This technique is called gradient descent.

In [44]:
# in backward propagation we use y_head thta found in forward propagation
# lets combine forward and backward propagation
def forward_backward_propagation(w,b,x_train,y_train):
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]      # x_train.shape[1]  is for scaling
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients

Initializing parameters (implemented)

Finding cost with forward propagation and cost function (implemented)

Updating(learning) parameters (weight and bias). Now lets implement it.

In [45]:
# Updating(learning) parameters
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
    # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list
#parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate = 0.009,number_of_iterarion = 200)

In [46]:
 # prediction
def predict(w,b,x_test):
    # x_test is a input for forward propagation
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    # if z is bigger than 0.5, our prediction is sign one (y_head=1),
    # if z is smaller than 0.5, our prediction is sign zero (y_head=0),
    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction
# predict(parameters["weight"],parameters["bias"],x_test)

In [47]:
# put them all together

     
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):
    # initialize
    dimension =  x_train.shape[0]  # that is 4096
    w,b = initialize_weights_and_bias(dimension)
    # do not change learning rate
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)
    y_prediction_train = predict(parameters["weight"],parameters["bias"],x_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
    
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 0.01, num_iterations = 150)